In [71]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy

In [72]:
#load all the trained model (ann , scaler, one_hot)
model = load_model('model.h5')

#encoder and scaler
with open('label_encoder_gender.pkl' , 'rb')as file :
    label_encoder_gender  = pickle.load(file)
    
with open('onehot_encoder_geo.pkl' , 'rb')as file :
    onehot_encoder_geo  = pickle.load(file)
    
with open('scaler.pkl' , 'rb')as file :
    scaler  = pickle.load(file)

In [118]:
# example data
input_data = {'CreditScore': 619,
 'Geography': 'France',
 'Gender': 'Female',
 'Age': 42,
 'Tenure': 2,
 'Balance': 0.0,
 'NumOfProducts': 1,
 'HasCrCard': 1,
 'IsActiveMember': 1,
 'EstimatedSalary': 101348.88,
 }  

# input_data = {'CreditScore': 608,
#  'Geography': 'Spain',
#  'Gender': 'Female',
#  'Age': 41,
#  'Tenure': 1,
#  'Balance': 83807.86,
#  'NumOfProducts': 1,
#  'HasCrCard': 0,
#  'IsActiveMember': 1,
#  'EstimatedSalary': 112542.58,
#  }


    

In [119]:
#we have to convert this category to numeric feature (for predict the data)

geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded , columns= onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df


/Users/anuhyasamudrala/Documents/Anu_uncc/Deeplearning/ANNClassification/venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [120]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.0,1,1,1,101348.88


In [121]:
#combine one hot encoding 
input_data = pd.concat([input_df.reset_index(drop = True) , geo_encoded_df] , axis =1)

In [122]:
##gender label encoder to categorical 
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,0,42,2,0.0,1,1,1,101348.88


In [123]:
#concat
input_df = pd.concat([input_df.drop("Geography", axis =1) , geo_encoded_df] , axis =1 )

In [124]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.0,1,1,1,101348.88,1.0,0.0,0.0


In [125]:
#scaling 
input_scaled = scaler.transform(input_df)
input_scaled 

array([[-0.33880827, -1.09499335,  0.29493847, -1.04241787, -1.21847056,
        -0.91668767,  0.64920267,  0.97481699,  0.01595384,  1.00150113,
        -0.57946723, -0.57638802]])

In [126]:
##predict churn 
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 10ms/step


array([[0.06713686]], dtype=float32)

In [134]:
prediction_prob = prediction[0][0]

In [136]:
if prediction_prob > 0.3:
    print('customer is not likely to stay in the bank/likes to churn')
else :
    print('customer is likely to stay in the bank')

customer is likely to stay in the bank
